In [3]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer

from hashlib import sha256
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf

pd.set_option('display.max_columns', None)

output = join("output","correcoes","tb_pacientes")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [5]:
timer.reset()
tb = TbPacientes()
tb.update()
timer.stop()
len(tb)

384393

In [ ]:
# timer.reset()
# casos_confirmados = CasosConfirmados()
# casos_confirmados.update()
# timer.stop()

In [6]:
# casost = tb.get()
# casosc = casos_confirmados.get_casos()
# print(f"casos casos_confirmados: {casosc.shape}")

casost = tb.fix_mun_resid_casos()
print(f"casos tb_pacientes: {casost.shape}")
casost.loc[(casost['ibge_res_pr'].isnull()) & (casost['mun_resid'].notnull()), 'mun_resid'] = casost.loc[(casost['ibge_res_pr'].isnull()) & (casost['mun_resid'].notnull()), 'mun_resid'] + '/' + casost.loc[(casost['ibge_res_pr'].isnull()) & (casost['mun_resid'].notnull()), 'uf_resid']
casost.loc[casost['ibge_res_pr'].isnull(),'ibge_res_pr'] = '9999999'
casost.to_excel(join(output,'normalize_mun.xlsx'))
exit()

KeyError: 'ibge_res_pr'

In [8]:
# casosc.loc[(casosc['rs'].isnull()) & (casosc['mun_resid'].notnull()), 'mun_resid'] = casosc.loc[(casosc['rs'].isnull()) & (casosc['mun_resid'].notnull()), 'mun_resid'] + '/' + casosc.loc[(casosc['rs'].isnull()) & (casosc['mun_resid'].notnull()), 'uf_resid']
casosc['hash'] = casosc.apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
casosc['hash_less'] = casosc.apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']-1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
casosc['hash_more'] = casosc.apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']+1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
casosc['hash_atend'] = casosc.apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_atend']))).hexdigest(), axis=1)
casosc['hash_less_atend'] = casosc.apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']-1)+normalize_hash(row['mun_atend']))).hexdigest(), axis=1)
casosc['hash_more_atend'] = casosc.apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']+1)+normalize_hash(row['mun_atend']))).hexdigest(), axis=1)


NameError: name 'casosc' is not defined

In [ ]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_tb_pacientes = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')

In [ ]:
idx_tb_pacientes = casost[casost['hash'].isin(casosc['hash'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_less'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_more'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_casos_confirmados = casost.loc[ set(casost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')


In [ ]:
print(f"casos_confirmados_nao_tb_pacientes: {casos_confirmados_nao_tb_pacientes.shape}")
casos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'casos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_casos_confirmados: {tb_pacientes_nao_casos_confirmados.shape}")
tb_pacientes_nao_casos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_casos_confirmados.xlsx'), index=False)

In [ ]:
obitosc = casosc.loc[casosc['obito']=='SIM']
print(f"obitos casos_confirmados: {obitosc.shape}")
obitost = casost.loc[casost['obito']=='SIM']
print(f"obitos tb_pacientes: {obitosc.shape}")

In [ ]:
# obitosc.loc[(obitosc['rs'].isnull()) & (obitosc['mun_resid'].notnull()), 'mun_resid'] = obitosc.loc[(obitosc['rs'].isnull()) & (obitosc['mun_resid'].notnull()), 'mun_resid'] + '/' + obitosc.loc[(obitosc['rs'].isnull()) & (obitosc['mun_resid'].notnull()), 'uf_resid']
obitosc['hash'] = obitosc.apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
obitosc['hash_less'] = obitosc.apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']-1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
obitosc['hash_more'] = obitosc.apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']+1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)

In [ ]:
idx_obitos_confirmados = obitosc[obitosc['hash'].isin(obitost['hash'])].index.tolist()
idx_obitos_confirmados += obitosc[obitosc['hash_less'].isin(obitost['hash'])].index.tolist()
idx_obitos_confirmados += obitosc[obitosc['hash_more'].isin(obitost['hash'])].index.tolist()
idx_obitos_confirmados = set(idx_obitos_confirmados)
obitos_confirmados_nao_tb_pacientes = obitosc.loc[ set(obitosc.index.tolist()) - idx_obitos_confirmados ].sort_values('ordem')

In [ ]:
idx_tb_pacientes = obitost[obitost['hash'].isin(obitosc['hash'])].index.tolist()
idx_tb_pacientes += obitost[obitost['hash'].isin(obitosc['hash_less'])].index.tolist()
idx_tb_pacientes += obitost[obitost['hash'].isin(obitosc['hash_more'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_obitos_confirmados = obitost.loc[ set(obitost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')

In [ ]:
print(f"obitos_confirmados_nao_tb_pacientes: {obitos_confirmados_nao_tb_pacientes.shape}")
obitos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'obitos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_obitos_confirmados: {tb_pacientes_nao_obitos_confirmados.shape}")
tb_pacientes_nao_obitos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_obitos_confirmados.xlsx'), index=False)

In [ ]:
# writer = pd.ExcelWriter(join(output,"novos_tb_pacientes.xlsx"),
#                         engine='xlsxwriter',
#                         datetime_format='dd/mm/yyyy',
#                         date_format='dd/mm/yyyy')

# casost.sort_values('nome').to_excel(writer,index=None)

# writer.save()
# writer.close()